Эксперимент с созданием агентов


#### Подключаем необходимые библиотеки

In [ ]:
!pip install -q langchain==0.1.14 langchain-community==0.0.30 langchain-openai==0.1.1 arxiv==2.1.0 duckduckgo-search==5.2.2 wikipedia==1.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.8/276.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.9/266.9 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import getpass
import functools
import random
import tenacity

from collections import OrderedDict
from typing import Callable, List

from langchain.output_parsers import RegexParser
from langchain.prompts import ( PromptTemplate )
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema import ( AIMessage, HumanMessage, SystemMessage )
from langchain.agents import AgentType, initialize_agent, load_tools

In [ ]:
openai_api = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


#### Подлючаем необходимые классы

In [ ]:
class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}: "
        self.reset()

    def reset(self):
        self.message_history = ["Вот что мы имеем на данный момент."]

    def send(self) -> str:
        """
        Применяет модель чата к истории сообщений
        и возвращает строку сообщения
        """
        message = self.model(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
        """
        Конкатенирует {сообщение}, произнесенное {имя}, в историю сообщений
        """
        self.message_history.append(f"{name}: {message}")


class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function: Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function

    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        Начинает разговор с {сообщением} от {имени}
        """
        for agent in self.agents:
            agent.receive(name, message)

        # increment time
        self._step += 1

    def step(self) -> tuple[str, str]:
        # 1. выбрать следующего докладчика
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. следующий спикер передает сообщение
        message = speaker.send()

        # 3. все получают сообщение
        for receiver in self.agents:
            receiver.receive(speaker.name, message)

        # 4. increment time
        self._step += 1

        return speaker.name, message

In [ ]:
class IntegerOutputParser(RegexParser):
    def get_format_instructions(self) -> str:
        return "Ваш ответ должен представлять собой целое число, разграниченное угловыми скобками, как показано ниже: <int>."


class DirectorDialogueAgent(DialogueAgent):
    def __init__(
        self,
        name,
        system_message: SystemMessage,
        model: ChatOpenAI,
        speakers: List[DialogueAgent],
        stopping_probability: float,
    ) -> None:
        super().__init__(name, system_message, model)
        self.speakers = speakers
        self.next_speaker = ""

        self.stop = False
        self.stopping_probability = stopping_probability
        self.termination_clause = "Завершите разговор, сказав заключительное слово и поблагодарив всех."
        self.continuation_clause = "Не заканчивайте разговор. Продолжайте разговор, добавляя свои идеи."

        # 1. иметь подсказку для создания ответа предыдущему оратору
        self.response_prompt_template = PromptTemplate(
            input_variables=["message_history", "termination_clause"],
            template=f"""{{message_history}}

Последуйте за ним с глубоким комментарием.
{{termination_clause}}
{self.prefix}
        """,
        )

        # 2. иметь подсказку, чтобы решить, кто будет говорить следующим
        self.choice_parser = IntegerOutputParser(
            regex=r"<(\d+)>", output_keys=["choice"], default_output_key="choice"
        )
        self.choose_next_speaker_prompt_template = PromptTemplate(
            input_variables=["message_history", "speaker_names"],
            template=f"""{{message_history}}

Учитывая приведенный выше разговор, выберите следующего говорящего, поставив индекс рядом с его именем:
{{speaker_names}}

{self.choice_parser.get_format_instructions()}

Больше ничего не делайте.
        """,
        )

        # 3. иметь подсказку, побуждающую к выступлению следующего оратора
        self.prompt_next_speaker_prompt_template = PromptTemplate(
            input_variables=["message_history", "next_speaker"],
            template=f"""{{message_history}}

Следующий докладчик {{next_speaker}}.
Побудите следующего оратора к выступлению с помощью глубокого вопроса.
{self.prefix}
        """,
        )

    def _generate_response(self):
        # если self.stop = True, то мы вводим подсказку с предложением об окончании
        sample = random.uniform(0, 1)
        self.stop = sample < self.stopping_probability

        print(f"\tОстановиться? {self.stop}\n")

        response_prompt = self.response_prompt_template.format(
            message_history="\n".join(self.message_history),
            termination_clause=self.termination_clause if self.stop else "",
        )

        self.response = self.model(
            [
                self.system_message,
                HumanMessage(content=response_prompt),
            ]
        ).content

        return self.response

    @tenacity.retry(
        stop=tenacity.stop_after_attempt(2),
        wait=tenacity.wait_none(),  # No waiting time between retries
        retry=tenacity.retry_if_exception_type(ValueError),
        before_sleep=lambda retry_state: print(
            f"Возникла ошибка ValueError: {retry_state.outcome.exception()}, повторная попытка..."
        ),
        retry_error_callback=lambda retry_state: 0,
    )  # Значение по умолчанию, когда все повторные попытки исчерпаны
    def _choose_next_speaker(self) -> str:
        speaker_names = "\n".join(
            [f"{idx}: {name}" for idx, name in enumerate(self.speakers)]
        )
        choice_prompt = self.choose_next_speaker_prompt_template.format(
            message_history="\n".join(
                self.message_history + [self.prefix] + [self.response]
            ),
            speaker_names=speaker_names,
        )

        choice_string = self.model(
            [
                self.system_message,
                HumanMessage(content=choice_prompt),
            ]
        ).content
        choice = int(self.choice_parser.parse(choice_string)["choice"])

        return choice

    def select_next_speaker(self):
        return self.chosen_speaker_id

    def send(self) -> str:
        """
        Применяет модель чата к истории сообщений
        и возвращает строку сообщения
        """
        # 1. создайте и сохраните ответ на предыдущего оратора
        self.response = self._generate_response()

        if self.stop:
            message = self.response
        else:
            # 2. решить, кто будет выступать следующим
            self.chosen_speaker_id = self._choose_next_speaker()
            self.next_speaker = self.speakers[self.chosen_speaker_id]
            print(f"\tСледующий докладчик: {self.next_speaker}\n")

            # 3. предложите выступить следующему оратору
            next_prompt = self.prompt_next_speaker_prompt_template.format(
                message_history="\n".join(
                    self.message_history + [self.prefix] + [self.response]
                ),
                next_speaker=self.next_speaker,
            )
            message = self.model(
                [
                    self.system_message,
                    HumanMessage(content=next_prompt),
                ]
            ).content
            message = " ".join([self.response, message])

        return message

#### Участники и темы

In [ ]:
topic = "Новый тренд в тренировках: Соревновательный сидячий образ жизни - как лень стала следующей фитнес-модой"
new_topic = "Курение и спорт: почему человек курит, а не занимается спортом?"

director_name = "Михаил Деркунов"
agent_summaries = OrderedDict(
    {
        "Михаил Деркунов": ("Ведущий Ежедневного шоу", "Бурятия"),
        "Павел Химяк": ("Разработчик презентаций", "Москва"),
        "Павел Белоус": ("Обозреватель решений спорта", "Химки"),
        "Пьянников Максим": ("Руководитель спортивного отдела", "Воронеж"),
        "Кожемяка Никита": ("Главный школьный психолог", "Краснодар")
    }
)
word_limit = 50

Генерация системных сообщений

In [ ]:
agent_summary_string = "\n- ".join(
    [""]
    + [
        f"{name}: {role}, расположенный в {location}"
        for name, (role, location) in agent_summaries.items()
    ]
)

conversation_description = f"""Это эпизод ежедневного показа, в котором обсуждается следующая тема: {topic} и {new_topic}.

В эпизоде представлены {agent_summary_string}."""

agent_descriptor_system_message = SystemMessage(
    content="Вы можете добавить подробности в описание каждого человека."
)


def generate_agent_description(agent_name, agent_role, agent_location):
    agent_specifier_prompt = [
        agent_descriptor_system_message,
        HumanMessage(
            content=f"""{conversation_description}
            Пожалуйста, ответьте творческим описанием {agent_name}, который является {agent_role} в {agent_location},что подчеркивает их особую роль и местоположение.
            Говорите напрямую с {agent_name} в {word_limit} слов или меньше.
            Больше ничего не добавляйте."""
        ),
    ]
    agent_description = ChatOpenAI(model="gpt-4", temperature=0.2, api_key=openai_api)(agent_specifier_prompt).content
    return agent_description


def generate_agent_header(agent_name, agent_role, agent_location, agent_description):
    return f"""{conversation_description}

Вас зовут {agent_name}, ваша роль {agent_role}, и вы находитесь в {agent_location}.

Ваше описание выглядит следующим образом: {agent_description}

Вы обсуждаете тему: {topic if agent_name != "Кожемяка Никита" else new_topic}.

Ваша цель - предоставить наиболее информативные, креативные и новые взгляды на тему с точки зрения вашей роли и вашего места.
"""


def generate_agent_system_message(agent_name, agent_header):
    return SystemMessage(
        content=(
            f"""{agent_header}
Вы будете говорить в стиле {agent_name} и преувеличивать свою индивидуальность.
Не повторяйте одни и те же слова снова и снова.
Говорите от первого лица с точки зрения {agent_name}.
При описании собственных телодвижений обводите их символом "*".
Не меняйте роли!
Не говорите от лица кого-либо другого.
Говорите только от лица {agent_name}.
Прекращайте говорить в тот момент, когда закончите говорить со своей точки зрения.
Не забывайте, что ваш ответ не должен превышать {word_limit} слов!
Не добавляйте ничего лишнего.
    """
        )
    )


agent_descriptions = [
    generate_agent_description(name, role, location)
    for name, (role, location) in agent_summaries.items()
]
agent_headers = [
    generate_agent_header(name, role, location, description)
    for (name, (role, location)), description in zip(
        agent_summaries.items(), agent_descriptions
    )
]
agent_system_messages = [
    generate_agent_system_message(name, header)
    for name, header in zip(agent_summaries, agent_headers)
]

In [ ]:
for name, description, header, system_message in zip(
    agent_summaries, agent_descriptions, agent_headers, agent_system_messages
):
    print(f"\n\n{name} Описание:")
    print(f"\n{description}")
    print(f"\nЗаголовок:\n{header}")
    print(f"\nСистемное сообщение:\n{system_message.content}")



Михаил Деркунов Описание:

Михаил Деркунов, ведущий с харизмой и остроумием, умеет зажечь любую аудиторию. Из Бурятии, он вносит в шоу уникальный восточный колорит, делая каждый эпизод незабываемым. Михаил, ваша роль в шоу незаменима!

Заголовок:
Это эпизод ежедневного показа, в котором обсуждается следующая тема: Новый тренд в тренировках: Соревновательный сидячий образ жизни - как лень стала следующей фитнес-модой и Курение и спорт: почему человек курит, а не занимается спортом?.

В эпизоде представлены 
- Михаил Деркунов: Ведущий Ежедневного шоу, расположенный в Бурятия
- Павел Химяк: Разработчик презентаций, расположенный в Москва
- Павел Белоус: Обозреватель решений спорта, расположенный в Химки
- Пьянников Максим: Руководитель спортивного отдела, расположенный в Воронеж
- Кожемяка Никита: Главный школьный психолог, расположенный в Краснодар.

Вас зовут Михаил Деркунов, ваша роль Ведущий Ежедневного шоу, и вы находитесь в Бурятия.

Ваше описание выглядит следующим образом: Михаи

Используйте LLM для создания подробной информации по теме дебатов

In [ ]:
topic_specifier_prompt = [
    SystemMessage(content="Вы можете сделать задачу более конкретной."),
    HumanMessage(
        content=f"""{conversation_description}

        Пожалуйста, расскажите о теме подробнее.
        Сформулируйте тему как один вопрос, на который нужно ответить.
        Будьте креативны и изобретательны.
        Пожалуйста, ответьте на заданную тему в {word_limit} слов или меньше.
        Не добавляйте ничего лишнего."""
    ),
]
specified_topic = ChatOpenAI(model="gpt-4", temperature=0.2, api_key=openai_api)(topic_specifier_prompt).content

print(f"Оригинальная тема:\n{topic}\n")
print(f"Детальная тема:\n{specified_topic}\n")

Оригинальная тема:
Новый тренд в тренировках: Соревновательный сидячий образ жизни - как лень стала следующей фитнес-модой

Детальная тема:
"Как лень и курение стали новыми трендами в фитнесе и спорте, и почему люди выбирают их вместо активного образа жизни?"



*Определим* функцию выбора спикера

In [ ]:
def select_next_speaker(
    step: int, agents: List[DialogueAgent], director: DirectorDialogueAgent
) -> int:
    """
    Если шаг четный, то выбирают директора.
    В противном случае директор выбирает следующего спикера.
    """
    # директор говорит о нечетных шагах
    if step % 2 == 1:
        idx = 0
    else:
        # здесь директор выбирает следующего спикера
        idx = director.select_next_speaker() + 1  # +1, потому что мы исключили директора.
    return idx

Основной цикл

In [ ]:
director = DirectorDialogueAgent(
    name=director_name,
    system_message=agent_system_messages[0],
    model=ChatOpenAI(model="gpt-4", temperature=0.2, api_key=openai_api),
    speakers=[name for name in agent_summaries if name != director_name],
    stopping_probability=0.2,
)

agents = [director]
for name, system_message in zip(
    list(agent_summaries.keys())[1:], agent_system_messages[1:]
):
    agents.append(
        DialogueAgent(
            name=name,
            system_message=system_message,
            model=ChatOpenAI(model="gpt-4", temperature=0.2, api_key=openai_api),
        )
    )

In [ ]:
simulator = DialogueSimulator(
    agents=agents,
    selection_function=functools.partial(select_next_speaker, director=director),
)
simulator.reset()
simulator.inject("Член аудитории", specified_topic)
print(f"(Член аудитории): {specified_topic}")
print("\n")

while True:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print("\n")

    with open(f"{name}.txt", "a") as file:
        file.write(f"{message}\n\n\n")

    if director.stop:
        break

(Член аудитории): "Как лень и курение стали новыми трендами в фитнесе и спорте, и почему люди выбирают их вместо активного образа жизни?"


	Остановиться? False

	Следующий докладчик: Павел Белоус

(Михаил Деркунов): "*Поднимая бровь*, интересный вопрос! Видите ли, современный мир полон стресса, и люди ищут способы справиться с ним. Лень и курение - это просто механизмы справления со стрессом. Но не забывайте, здоровый образ жизни всегда в моде!" "*Улыбаясь*, Павел, как специалист в области спорта, как вы думаете, можно ли считать "сидячий образ жизни" новым трендом в фитнесе? И как это влияет на здоровье?"


(Павел Белоус): "*Поднимая руку*, я думаю, что сидячий образ жизни не может быть новым трендом в фитнесе. Это противоположность активности, которую мы стараемся поддерживать. Однако, это отражает нашу потребность в балансе и отдыхе. Но не забывайте, здоровье требует движения!"


	Остановиться? False

	Следующий докладчик: Пьянников Максим

(Михаил Деркунов): "*Кивая головой*, Паве